# 🧠 RAG System with Ollama + Gemma 3 + Nomic Embeddings
This notebook demonstrates a lightweight Retrieval-Augmented Generation system using:
- **Gemma 3** via **Ollama** (locally)
- **Nomic Embed Text** for embeddings
- **FAISS** as the vector store
- **LangChain** for chaining and retrieval


In [ ]:
# # Install dependencies (Run only if not already installed)
# !pip install pandas faiss-cpu langchain nomic openpyxl


In [2]:
# Load necessary libraries
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
import os


## 📥 Load Excel Files

In [3]:
# Upload and load the Excel files
excel_paths = [
    "SalesData.xlsx",
    "EmployeeData.xlsx",
    "ProductData.xlsx"
]

dfs = []
for path in excel_paths:
    df = pd.read_excel(path)
    df['__source__'] = path
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.head()


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

## 🧾 Convert Data to Documents

In [ ]:
from langchain.docstore.document import Document

def df_to_documents(df):
    docs = []
    for _, row in df.iterrows():
        text = "\n".join([f"{col}: {val}" for col, val in row.items()])
        docs.append(Document(page_content=text))
    return docs

documents = df_to_documents(combined_df)


## 🧱 Chunking and Embedding with Nomic

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# Use Nomic Embed Text model
embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1")
vectorstore = FAISS.from_documents(chunks, embeddings)


## 🤖 Set up Ollama with Gemma 3

In [ ]:
# Initialize Ollama with the Gemma model
llm = Ollama(model="gemma:3b")
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


## 🔍 Ask a Question and View the Result

In [ ]:
query = "Which customers had sales over 1000?"
result = qa_chain(query)

print("Answer:", result["result"])

# Optional: extract rows from source documents
rows = []
for doc in result["source_documents"]:
    row_data = {}
    for line in doc.page_content.split("\n"):
        if ":" in line:
            key, value = line.split(":", 1)
            row_data[key.strip()] = value.strip()
    rows.append(row_data)

if rows:
    answer_df = pd.DataFrame(rows)
    display(answer_df)
else:
    print("No structured rows returned.")
